In [1]:
import dataloader
import numpy as np
import torch
import random
import scipy.signal as sig

In [2]:
raw_dataset = dataloader.get_dataset(None, 8192)
print({ k: len(v) for k, v in raw_dataset.items() })

{'Whistling': 22, 'Train': 895, 'Clapping': 546, 'BackgroundSounds': 8192, 'Laughing': 216, 'Whistle': 21, 'Horn': 1146, 'Dog': 2768, 'Noise': 1950, 'Rooster': 402, 'BirdChirp': 1299, 'Crow': 621, 'Scream': 36, 'Siren': 2363, 'Sneezing': 154, 'Aircraft': 1119, 'Gunshot': 8, 'Wind': 1408, 'Frog': 3608, 'CarDoor': 452, 'VehicleExhaust': 292, 'Engine': 5845, 'Coughing': 736, 'Cat': 242, 'Thunder': 488, 'Beeping': 98, 'Bell': 702, 'Unknown': 293, 'Insects': 1986, 'Sheep': 191, 'Basketball': 752, 'Drums': 6193, 'Cow': 168, 'GunNoise': 2448, 'PowerTool': 1537, 'GlassBreak': 982, 'Skateboarding': 76, 'Jackhammer': 2882, 'HandSaw': 332}


In [3]:
print(f'gpu enabled: {torch.cuda.is_available()}')
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

# import vae_7 as vae
# encoder = vae.Encoder(embedding_size = 4).to(device)
# encoder.load_state_dict(torch.load('vae-7-F4-tested-55/encoder-F4-E200-L110.pt')) # 85/50 ish

# import vae_8 as vae
# encoder = vae.Encoder(embedding_size = 4).to(device)
# encoder.load_state_dict(torch.load('vae-8-F4-A0.50-tested-54/encoder-F4-A0.50-E208-L56.pt')) # 80/50 ish
# encoder.load_state_dict(torch.load('vae-8-F4-A0.33-tested-44/encoder-F4-A0.33-E200-L41.pt')) # 85/60 ish
# encoder.load_state_dict(torch.load('vae-8-F4-A0.67-tested-57/encoder-F4-A0.67-E224-L71.pt')) # 85/50 ish

# import vae_8 as vae
# encoder = vae.Encoder(embedding_size = 16).to(device)
# encoder.load_state_dict(torch.load('vae-8-F16-A0.50-tested-46/encoder-F16-A0.50-E200-L57.pt')) # 85/60 ish
# encoder.load_state_dict(torch.load('vae-8-F16-A0.33-tested-42/encoder-F16-A0.33-E256-L43.pt')) # 85/50 ish
# encoder.load_state_dict(torch.load('vae-8-F16-A0.9-tested-56/encoder-F16-A0.9-E256-L89.pt')) # 85/50 ish

# import vae_9 as vae
# encoder = vae.Encoder(embedding_size = 16).to(device)
# encoder.load_state_dict(torch.load('vae-9-F16-A0.9-tested-55/encoder-F16-A0.9-E256-L83.pt')) # 85/55 ish

# import vae_10 as vae
# encoder = vae.Encoder(embedding_size = 16).to(device)
# encoder.load_state_dict(torch.load('vae-10-F16-A0.9-tested-60-79/encoder-F16-A0.9-E256-L66.pt')) # 85/55 ish

import vae_11 as vae
encoder = vae.Encoder(embedding_size = 16).to(device)
# encoder.load_state_dict(torch.load('vae-11-F16-A3150.0:10.0:1.0-meta-85-45/encoder-F16-A3150.0:10.0:1.0-E256-L2489.pt'))
# encoder.load_state_dict(torch.load('vae-11-F16-A3150.0:10.0:2.0-meta-85-55/encoder-F16-A3150.0:10.0:2.0-E256-L2420.pt'))
# encoder.load_state_dict(torch.load('vae-11-F16-A1.0:1.0:1.0-meta-90-50/encoder-F16-A1.0:1.0:1.0-E256-L855.pt'))
encoder.load_state_dict(torch.load('untested-5/encoder-F16-A1.0:1.0:0.0-E256-L2394.pt'))

encoder.eval()

def prep_sample(x):
    s, _ = dataloader.get_spectrogram_normalized(x)
    with torch.no_grad():
        encoder_input = torch.tensor(s.reshape(1, *s.shape)).to(device)
        mean, logstd = encoder.forward(encoder_input)
        assert mean.shape == (1, encoder.embedding_size)
        return np.array(mean.cpu()).reshape(-1), np.array(logstd.exp().cpu()).reshape(-1)

dataset = { k: [prep_sample(x) for x in v] for k, v in raw_dataset.items() }

def distance_metric(a, b):
    assert a.shape[-1:] == (16,)
    assert b.shape == (16,)

    return np.sum((a - b)**2, axis = -1)

gpu enabled: True


In [4]:
benchmark_dataset = dict(random.sample([(k,v) for k,v in dataset.items() if len(v) >= 256], 10))
print({ k: len(v) for k, v in benchmark_dataset.items() })
max_clusters = 32
def benchmark(filter_class, filter_thresh):
    print(f'bench: {filter_class.__name__}')
    print()
    print('heterogeneous class tests:')
    vals = []
    for i in range(16):
        f = filter_class(max_clusters, encoder.embedding_size, filter_thresh)
        keeps = []
        reps = 30
        for rep in range(reps):
            for label, samples in benchmark_dataset.items():
                if f.insert(*random.choice(samples)):
                    keeps.append(label)
        vals.append(100 * len(keeps) / len(benchmark_dataset) / reps)
        print(f'iter {i:>4}: kept {len(keeps)}/{len(benchmark_dataset) * reps} ({vals[-1]:.2f}%)')
    print(f'mean: {np.mean(vals):>5.2f} std: {np.std(vals):>5.2f}')

    print()
    print('homogenous class tests:')
    vals = []
    for label, samples in benchmark_dataset.items():
        f = filter_class(max_clusters, encoder.embedding_size, filter_thresh)
        keeps = []
        reps = 300
        for rep in range(reps):
            if f.insert(*random.choice(samples)):
                keeps.append(i)
        vals.append(100 * len(keeps) / reps)
        print(f'{label:>16}: kept {len(keeps):>4}/{reps} ({vals[-1]:>5.2f}%)')
    print(f'mean: {np.mean(vals):>5.2f} std: {np.std(vals):>5.2f}')

{'Engine': 5845, 'Drums': 6193, 'Clapping': 546, 'Wind': 1408, 'Siren': 2363, 'BackgroundSounds': 8192, 'Crow': 621, 'GunNoise': 2448, 'Insects': 1986, 'GlassBreak': 982}


In [23]:
class ClusterFilterAug2:
    def __init__(self, max_clusters, embedding_size, thresh):
        self.means = np.zeros((max_clusters, embedding_size))
        self.weights = np.zeros((max_clusters,))
        self.max_clusters = max_clusters
        self.base_radius = thresh
    def insert(self, mean, std):
        assert mean.shape == self.means.shape[1:] and mean.shape == std.shape and self.means.shape[0] == self.max_clusters and self.weights.shape == (self.max_clusters,)
        l2_norm = np.sqrt(np.sum((self.means - mean) ** 2, axis = 1))
        close = l2_norm <= np.sqrt(self.weights) * self.base_radius
        
        if np.any(close):
            center = (np.sum((self.means[close].T * self.weights[close]).T, axis = 0) + mean) / (np.sum(self.weights[close]) + 1)
            weight = np.sum(self.weights[close]) + 1
            self.means = np.concatenate([
                self.means[~close],
                [ center ],
                np.zeros((self.means.shape[0] - (np.sum(~close) + 1), self.means.shape[1])),
            ])
            self.weights = np.concatenate([
                self.weights[~close],
                [ weight ],
                np.zeros((self.weights.shape[0] - (np.sum(~close) + 1),)),
            ])
            return False
        else:
            self.means = np.concatenate([
                self.means[1:],
                [ mean ],
            ])
            self.weights = np.concatenate([
                self.weights[1:],
                [ 1 ],
            ])
            return True

benchmark(ClusterFilterAug2, 0.5)

bench: ClusterFilterAug2

heterogeneous class tests:
iter    0: kept 274/300 (91.33%)
iter    1: kept 298/300 (99.33%)
iter    2: kept 281/300 (93.67%)
iter    3: kept 156/300 (52.00%)
iter    4: kept 291/300 (97.00%)
iter    5: kept 275/300 (91.67%)
iter    6: kept 281/300 (93.67%)
iter    7: kept 216/300 (72.00%)
iter    8: kept 295/300 (98.33%)
iter    9: kept 187/300 (62.33%)
iter   10: kept 288/300 (96.00%)
iter   11: kept 295/300 (98.33%)
iter   12: kept 293/300 (97.67%)
iter   13: kept 294/300 (98.00%)
iter   14: kept 296/300 (98.67%)
iter   15: kept 243/300 (81.00%)
mean: 88.81 std: 13.99

homogenous class tests:
          Engine: kept   41/300 (13.67%)
           Drums: kept  300/300 (100.00%)
        Clapping: kept  219/300 (73.00%)
            Wind: kept  295/300 (98.33%)
           Siren: kept  274/300 (91.33%)
BackgroundSounds: kept  200/300 (66.67%)
            Crow: kept  284/300 (94.67%)
        GunNoise: kept  177/300 (59.00%)
         Insects: kept   41/300 (13.67%)
 

In [57]:
class ClusterFilterAug:
    def __init__(self, max_clusters, embedding_size, thresh):
        self.means = np.zeros((max_clusters, embedding_size)) + np.inf
        self.radii = np.zeros((max_clusters,))
        self.max_clusters = max_clusters
        self.base_radius = thresh
    def insert(self, mean, std):
        assert mean.shape == self.means.shape[1:] and mean.shape == std.shape and self.means.shape[0] == self.max_clusters and self.radii.shape == (self.max_clusters,)
        l2_norm = np.sqrt(np.sum((self.means - mean) ** 2, axis = 1))
        close = l2_norm <= self.radii
        
        if np.any(close):
            center = np.mean(self.means[close], axis = 0)
            radius = np.sqrt(np.sum(self.radii[close]**2))

#             center = (np.sum(self.means[close], axis = 0) + mean) / (np.sum(close) + 1)
#             radius = np.sqrt(np.sum(self.radii[close]**2) + self.base_radius**2)
        
            self.means = np.concatenate([
                self.means[~close],
                [ center ],
                np.zeros((self.means.shape[0] - (np.sum(~close) + 1), self.means.shape[1])) + np.inf,
            ])
            self.radii = np.concatenate([
                self.radii[~close],
                [ radius ],
                np.zeros((self.radii.shape[0] - (np.sum(~close) + 1),)),
            ])
            return False
        else:
            self.means = np.concatenate([
                self.means[1:],
                [ mean ],
            ])
            self.radii = np.concatenate([
                self.radii[1:],
                [ self.base_radius ],
            ])
            return True

benchmark(ClusterFilterAug, .7)

bench: ClusterFilterAug

heterogeneous class tests:
iter    0: kept 280/300 (93.33%)
iter    1: kept 279/300 (93.00%)
iter    2: kept 264/300 (88.00%)
iter    3: kept 279/300 (93.00%)
iter    4: kept 275/300 (91.67%)
iter    5: kept 277/300 (92.33%)
iter    6: kept 273/300 (91.00%)
iter    7: kept 283/300 (94.33%)
iter    8: kept 262/300 (87.33%)
iter    9: kept 273/300 (91.00%)
iter   10: kept 279/300 (93.00%)
iter   11: kept 281/300 (93.67%)
iter   12: kept 277/300 (92.33%)
iter   13: kept 279/300 (93.00%)
iter   14: kept 269/300 (89.67%)
iter   15: kept 277/300 (92.33%)
mean: 91.81 std:  1.92

homogenous class tests:
       BirdChirp: kept  208/300 (69.33%)
BackgroundSounds: kept  255/300 (85.00%)
       PowerTool: kept   70/300 (23.33%)
         Rooster: kept  232/300 (77.33%)
         HandSaw: kept  195/300 (65.00%)
        GunNoise: kept  265/300 (88.33%)
             Dog: kept  287/300 (95.67%)
        Aircraft: kept   58/300 (19.33%)
           Train: kept   37/300 (12.33%)
   

In [7]:
class ClusterFilter:
    def __init__(self, max_clusters, embedding_size, thresh):
        self.means = np.zeros((max_clusters, embedding_size)) + np.inf
        self.pos = 0
        self.thresh = thresh
    def insert(self, mean, std):
        assert mean.shape == self.means.shape[1:] and mean.shape == std.shape
        l2_norm = np.sqrt(np.sum((self.means - mean) ** 2, axis = 1))
        keep = np.min(l2_norm) > self.thresh
        self.means[self.pos,:] = mean
        self.pos = (self.pos + 1) % self.means.shape[0]
        return keep

benchmark(ClusterFilter, 1.4)

bench: ClusterFilter

heterogeneous class tests:
iter    0: kept 230/300 (76.67%)
iter    1: kept 219/300 (73.00%)
iter    2: kept 214/300 (71.33%)
iter    3: kept 215/300 (71.67%)
iter    4: kept 238/300 (79.33%)
iter    5: kept 218/300 (72.67%)
iter    6: kept 233/300 (77.67%)
iter    7: kept 210/300 (70.00%)
iter    8: kept 212/300 (70.67%)
iter    9: kept 218/300 (72.67%)
iter   10: kept 214/300 (71.33%)
iter   11: kept 235/300 (78.33%)
iter   12: kept 210/300 (70.00%)
iter   13: kept 224/300 (74.67%)
iter   14: kept 228/300 (76.00%)
iter   15: kept 225/300 (75.00%)
mean: 73.81 std:  2.96

homogenous class tests:
        Aircraft: kept  117/300 (39.00%)
           Train: kept   91/300 (30.33%)
       PowerTool: kept  147/300 (49.00%)
      Jackhammer: kept  113/300 (37.67%)
BackgroundSounds: kept  226/300 (75.33%)
           Noise: kept  107/300 (35.67%)
          Engine: kept  165/300 (55.00%)
       BirdChirp: kept  236/300 (78.67%)
         CarDoor: kept  174/300 (58.00%)
      

In [8]:
class DistributionFilter:
    def __init__(self, max_clusters, embedding_size, thresh):
        self.means = np.zeros((max_clusters, embedding_size))
        self.stds = np.zeros((max_clusters, embedding_size))
        self.pos = 0
        self.thresh = thresh
    def insert(self, mean, std):
        assert mean.shape == self.means.shape[1:] and mean.shape == std.shape
        kl_div = np.sum(-0.5 * (1 + 2 * np.log(np.maximum(self.stds / std, 1e-20)) - (std**2 + (self.means - mean)**2) / std**2), axis = 1)
        keep = np.min(kl_div) > self.thresh
        self.means[self.pos,:] = mean
        self.stds[self.pos,:] = std
        self.pos = (self.pos + 1) % self.means.shape[0]
        return keep

benchmark(DistributionFilter, 200)

bench: DistributionFilter

heterogeneous class tests:
iter    0: kept 300/300 (100.00%)
iter    1: kept 299/300 (99.67%)
iter    2: kept 298/300 (99.33%)
iter    3: kept 299/300 (99.67%)
iter    4: kept 299/300 (99.67%)
iter    5: kept 300/300 (100.00%)
iter    6: kept 300/300 (100.00%)
iter    7: kept 300/300 (100.00%)
iter    8: kept 300/300 (100.00%)
iter    9: kept 299/300 (99.67%)
iter   10: kept 299/300 (99.67%)
iter   11: kept 297/300 (99.00%)
iter   12: kept 300/300 (100.00%)
iter   13: kept 300/300 (100.00%)
iter   14: kept 299/300 (99.67%)
iter   15: kept 300/300 (100.00%)
mean: 99.77 std:  0.28

homogenous class tests:
        Aircraft: kept  295/300 (98.33%)
           Train: kept  290/300 (96.67%)
       PowerTool: kept  293/300 (97.67%)
      Jackhammer: kept  293/300 (97.67%)
BackgroundSounds: kept  297/300 (99.00%)
           Noise: kept  297/300 (99.00%)
          Engine: kept  298/300 (99.33%)
       BirdChirp: kept  287/300 (95.67%)
         CarDoor: kept  274/300 (9